In [3]:
from sklearn.datasets import load_boston
import numpy as np
from easydict import EasyDict as edict
from pdb import set_trace
import numpy as np
from collections import OrderedDict

In [4]:
class CombLinTs:
    def __init__(self, features_dim, p_lambda, p_sigma, solver, oracle):
        self.d = features_dim
        self.p_lambda = p_lambda
        self.p_sigma = p_sigma
        self.sigma = (p_lambda ** 2) * np.eye(self.d)
        self.theta = np.zeros(self.d)
        self.solver = solver
        self.oracle = oracle
        
    def sample_theta(self):
        return np.random.multivariate_normal(self.theta, self.sigma)
    
    def update_params(self, features, weights):
        theta = self.theta
        sigma = self.sigma
        
        n = len(weights)
        
        for i in range(n):
            f_vec = np.expand_dims(features[i], axis=1)
            t1 = np.matmul(sigma, np.matmul(f_vec, f_vec.T))
            t2 = np.matmul(f_vec.T, np.matmul(sigma, f_vec)) + self.p_sigma ** 2
            t3 = np.matmul(sigma, f_vec)
            t4 = np.matmul(np.matmul(f_vec.T, sigma), f_vec) + self.p_sigma ** 2
            theta = np.matmul((np.eye(sigma.shape[0]) - t1 / t2), theta) + \
                        np.squeeze(t3 / t4) * weights[i]

            t1 = np.matmul(np.matmul(sigma, np.matmul(f_vec, f_vec.T)), sigma)
            t2 = np.matmul(np.matmul(f_vec.T, sigma), f_vec) + self.p_sigma ** 2
            sigma = sigma - t1/t2

        self.theta = theta
        self.sigma = sigma
            
    def bandit_iter(self, problem):
        theta = self.sample_theta()
        At = self.solver.solve(problem, np.matmul(problem.features, theta))
        wAt = self.oracle.observe(problem, At)
        self.update_params(problem.features[At], wAt)
        
    def solve(self, problem):
        theta = self.sample_theta()
        return self.solver.solve(problem, np.matmul(problem.features, theta))

In [5]:
class Oracle:
    def observe(self, problem_info, solution):
        pass
    
class Solver:
    def solve(self, problem_info, weights):
        pass

In [6]:
problem_info = load_boston()
problem = edict({'problem_info': problem_info, 'features': problem_info["data"]})

In [7]:
class BostenSolver(Solver):
    def solve(self, problem, weights):
        return (-weights).argsort()[:50]
    
class BostenOracle(Oracle):
    def observe(self, problem, solution):
        return problem.problem_info.target[solution]
    

In [8]:
bs = BostenSolver()
bo = BostenOracle()


In [9]:
At = bs.solve(problem, problem_info.target)
print(At)
print(bo.observe(problem, At))
print(np.sum(bo.observe(problem, At)))

[204 267 257 225 195 368 369 370 371 372 283 186 163 161 166 162 262 203
 233 228 282 280 224 256  98 268 261 253 202 232 157 180  97 182 226 291
 179 190 264 192   4 181 304 258 281  55 273 279 189 199]
[50.  50.  50.  50.  50.  50.  50.  50.  50.  50.  50.  50.  50.  50.
 50.  50.  48.8 48.5 48.3 46.7 46.  45.4 44.8 44.  43.8 43.5 43.1 42.8
 42.3 41.7 41.3 39.8 38.7 37.9 37.6 37.3 37.2 37.  36.5 36.4 36.2 36.2
 36.1 36.  35.4 35.4 35.2 35.1 34.9 34.9]
2164.8


In [10]:
p_lambda = 10.
p_sigma = 0.1
features_dim = problem_info['data'].shape[1]
bandit = CombLinTs(features_dim=features_dim, p_lambda=p_lambda,
                     p_sigma=p_sigma, solver=bs, oracle=bo)

In [11]:
for i in range(10):
    bandit.bandit_iter(problem=problem)
    if i % 2 == 0:
        At = bandit.solve(problem)
        print np.sum(bo.observe(problem, At))

984.9000000000001
2050.7999999999997
2058.3
2063.7000000000003
2063.7000000000003


/home/halshater/anaconda/envs/gc_vision/lib/python2.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: covariance is not positive-semidefinite.
  if sys.path[0] == '':


In [12]:
print ("Learnt")
At = bandit.solve(problem)
print(np.sum(bo.observe(problem, At)))

print
print ("Actual")
At = bs.solve(problem, problem_info.target)
print(np.sum(bo.observe(problem, At)))

Learnt
2063.7000000000003

Actual
2164.8


In [13]:
import numpy as np


In [14]:
#https://people.eecs.berkeley.edu/~jordan/courses/260-spring10/lectures/lecture5.pdf

class GaussianLearner:
    def __init__(self, mu0, n0, tau0, alpha0, beta0):
        self.params = (float(mu0), float(n0), float(alpha0), float(beta0))
    def update_batch(self, x):
        mu0, v, alpha, beta = self.params
        n = len(x)
        xbar = np.mean(x)
        self.params = ((v * mu0 + n * xbar) / (v + n),
            v + n,
            alpha + n /2, 
            beta + 0.5 * np.sum((x - xbar) ** 2) + (n * v) / (n + v) * (xbar - mu0) ** 2 / 2)
    def sample(self):
        mu, std = self.sample_params()
        return np.random.normal(mu, std)
    def sample_params(self):
        mu0, v, alpha, beta = self.params
        tau_sample = np.random.gamma(alpha, 1. / beta) 
        mu_sample = np.random.normal(mu0, np.sqrt(1. / (v * tau_sample)))
        return mu_sample, np.sqrt(1. / (tau_sample))

In [15]:
class BernoulliLearner:
    def __init__(self, alpha0, beta0):
        self.params = (alpha0, beta0)
    
    def update_batch(self, x):
        xp = np.sum(x==1)
        xn = np.sum(x==0)
        alpha0, beta0 = self.params
        #set_trace()
        self.params = (alpha0 + xp, beta0 + xn)
        
    def sample(self):
        alpha, beta = self.params
        theta_sample = np.random.beta(alpha, beta) 
        return np.random.binomial(1, theta_sample)

In [16]:
bl = BernoulliLearner(1.0, 1.0)
bl.update_batch(np.array([1, 0, 0, 0, 1, 0, 1]*1000))
bl.sample()

0

In [17]:
gl = GaussianLearner(0.0, 2.0, 1.0, .02, .02)
gl.update_batch(np.array([5, 3.0, 3, 4, 1, 4, 10]*1000))
gl.sample()

4.285484709513354

In [18]:
gl = GaussianLearner(0, 10, 1, 2.0, 2.0)
for i in [5, 3.0, 3, 4, 1, 4, 10]*1000:
    gl.update_batch(np.array([i]))
gl.sample()

4.468704464594442

In [19]:
class Bandit:
    def __init__(self, arm_class, arm_init_params, n_arms):
        self.arms = [arm_class(*arm_init_params) for arm in range(n_arms)]
    
    def thompson_sample(self):
        samples = np.array([self.arms[i].sample() for i in range(len(self.arms))])
        sample = np.random.choice(np.where(samples[np.argmax(samples)] == samples)[0], 1)[0]
        return (sample, samples[sample])
    
    def sumbit_feedback(self, arm, result_array):
        self.arms[arm].update_batch(result_array)


In [20]:
b = Bandit(GaussianLearner, (0, 2, 0.01, 0.2, 0.2), 6)

In [21]:
gt = np.array([0, 2, 7.9, -3, 8.2, 8.004])

In [22]:
for i in range(100):
    arm, val = b.thompson_sample()
    #print arm, val
    b.sumbit_feedback(arm, np.array([gt[arm] + (np.random.random() - 0.5) * 4]))

In [23]:
print b.arms[2].mu_params
print b.arms[4].mu_params
print b.arms[5].mu_params

(7.160752822989986, 22.0, 50.99999999999999)
(7.998961980904404, 66.0, 161.0)
(6.022395230796314, 10.0, 21.0)


In [24]:
b = Bandit(BernoulliLearner, (5.0, 1.), 6)
gt = np.array([0.9, 0.8, 0.3, 0.01, 0.5, .004])

for i in range(1000):
    arm, val = b.thompson_sample()
    #print arm, val
    b.sumbit_feedback(arm, np.array(np.random.binomial(1, gt[arm])))

In [25]:
for i in range(6):
    print b.arms[i].params[0] / np.sum(b.arms[i].params)
    #print b.arms[i].params

0.8872832369942196
0.7669172932330827
0.32558139534883723
0.10204081632653061
0.5212765957446809
0.08620689655172414


In [26]:
b.arms[1].params

(204.0, 62.0)

In [27]:
# TODO
# How do learning occure ?
# plot hoeffding vs eperical error
# A/B testing
# Beta-Ber bandit
# Gaussian bandit
# LinUCB

# CombLinTs
# Shortest path 
# Mathcing (constraint satisfaction)

# What if we even do not have a problem structure at all ?!

In [28]:
# 10 workers, each 5 tasks, work can flow given certain network structuer

# request (a, b, c, d, e)

# weight 
# brockalbe 
# fresh
# size

# worker 
# skills



In [29]:
# from https://github.com/elemel/python-astar/blob/master/src/astar.py
from heapq import heappush, heappop
from sys import maxint

F, H, NUM, G, POS, OPEN, VALID, PARENT = xrange(8)

def astar(start_pos, neighbors, goal, start_g, cost, heuristic, limit=maxint,
          debug=None):

    """Find the shortest path from start to goal.
    Arguments:
      start_pos      - The starting position.
      neighbors(pos) - A function returning all neighbor positions of the given
                       position.
      goal(pos)      - A function returning true given a goal position, false
                       otherwise.
      start_g        - The starting cost.
      cost(a, b)     - A function returning the cost for moving from one
                       position to another.
      heuristic(pos) - A function returning an estimate of the total cost
                       remaining for reaching goal from the given position.
                       Overestimates can yield suboptimal paths.
      limit          - The maximum number of positions to search.
      debug(nodes)   - This function will be called with a dictionary of all
                       nodes.
    The function returns the best path found. The returned path excludes the
    starting position.
    """

    # Create the start node.
    nums = iter(xrange(maxint))
    start_h = heuristic(start_pos)
    start = [start_g + start_h, start_h, nums.next(), start_g, start_pos, True,
             True, None]

    # Track all nodes seen so far.
    nodes = {start_pos: start}

    # Maintain a heap of nodes.
    heap = [start]

    # Track the best path found so far.
    best = start

    while heap:

        # Pop the next node from the heap.
        current = heappop(heap)
        current[OPEN] = False

        # Have we reached the goal?
        if goal(current[POS]):
            best = current
            break

        # Visit the neighbors of the current node.
        for neighbor_pos in neighbors(current[POS]):
            neighbor_g = current[G] + cost(current[POS], neighbor_pos)
            neighbor = nodes.get(neighbor_pos)
            if neighbor is None:

                # Limit the search.
                if len(nodes) >= limit:
                    continue

                # We have found a new node.
                neighbor_h = heuristic(neighbor_pos)
                neighbor = [neighbor_g + neighbor_h, neighbor_h, nums.next(),
                            neighbor_g, neighbor_pos, True, True, current[POS]]
                nodes[neighbor_pos] = neighbor
                heappush(heap, neighbor)
                if neighbor_h < best[H]:

                    # We are approaching the goal.
                    best = neighbor

            elif neighbor_g < neighbor[G]:

                # We have found a better path to the neighbor.
                if neighbor[OPEN]:

                    # The neighbor is already open. Finding and updating it
                    # in the heap would be a linear complexity operation.
                    # Instead we mark the neighbor as invalid and make an
                    # updated copy of it.

                    neighbor[VALID] = False
                    nodes[neighbor_pos] = neighbor = neighbor[:]
                    neighbor[F] = neighbor_g + neighbor[H]
                    neighbor[NUM] = nums.next()
                    neighbor[G] = neighbor_g
                    neighbor[VALID] = True
                    neighbor[PARENT] = current[POS]
                    heappush(heap, neighbor)

                else:

                    # Reopen the neighbor.
                    neighbor[F] = neighbor_g + neighbor[H]
                    neighbor[G] = neighbor_g
                    neighbor[PARENT] = current[POS]
                    neighbor[OPEN] = True
                    heappush(heap, neighbor)

        # Discard leading invalid nodes from the heap.
        while heap and not heap[0][VALID]:
            heappop(heap)

    if debug is not None:
        # Pass the dictionary of nodes to the caller.
        debug(nodes)

    # Return the best path as a list.
    path = []
    current = best
    while current[PARENT] is not None:
        path.append(current[POS])
        current = nodes[current[PARENT]]
    path.reverse()
    return path

In [30]:
pathes = {'s': ['0', '1'],
          '1': ['2', '3', '4'],
          '0': ['2', '3', '4'],
          '2': ['7', '8'],
          '3': ['5', '6', '8'],
          '4': ['5', '6'],
          '5': ['9'],
          '6': ['10'],
          '7': ['9'],
          '8': ['9', '10'],
          '9': ['e'],
          '10': ['e']}

def neighbors(x):
    return pathes[x]

def build_weights(pathes):
    weights = {}
    for n, neighbours in pathes.items():
        for nbr in neighbours:
            weights[(n, nbr)] = np.random.uniform(0, 1, 1)[0]
    return weights

weights = build_weights(pathes)

def get_weight(x, y):
    return weights[(x, y)]

class ShortestPathProblem:
    def __init__(self):
        self.neighbors = neighbors
        self.get_weight = get_weight
        self.idx2arms = {str(k): v for (k, v) in enumerate(weights.keys())}
        self.arms2idx = {v: k for k, v in self.idx2arms.items()}
shortest_path_problem = ShortestPathProblem()

In [31]:
class ShortestPathSolver(Solver):
    def solve(self, problem, weights):
        At = astar('s', problem.neighbors, lambda x: x == 'e', 0, lambda x, y: weights(x, y), lambda x: 0, limit=maxint,
          debug=None) 
        return zip(['s'] + At[:-1], At)
    
class ShortestPathOracle(Oracle):
    def observe(self, problem, solution):
        return [problem.get_weight(x, y) for (x, y) in solution]
    

In [32]:
class CombUcb1:
    def __init__(self, problem, solver, oracle, mode='Max'):
        self.problem = problem
        self.solver = solver
        self.oracle = oracle
        self.arms = [str(i) for i in range(len(problem.arms2idx))]
        self.mode = mode
        self.init_algorithm()
        
    def init_algorithm(self):
        u = np.ones(len(self.arms))
        w = np.zeros(len(self.arms))
        t = 0
        for arm in range(len(self.arms)):
            if self.mode == 'Max':
                u[arm] = 1
            elif self.mode == 'Min':
                u[arm] = 0
            else:
                raise 'Mode is only Max or Min'
            w[arm] = 0
        solution_exists = True
        while solution_exists and np.sum(u) > 0: 
            At = self.solver.solve(self.problem, lambda x, y: u[int(self.problem.arms2idx[(x, y)])])
            if At is None:
                solution_exists = False
                break
            AtW = self.oracle.observe(self.problem, At)
            for idx, e in enumerate(At):
                e = int(self.problem.arms2idx[e])
                w[e] = AtW[idx]
                if self.mode == 'Max':
                    u[arm] = 0
                elif self.mode == 'Min':
                    u[arm] = 1
                else:
                    raise 'Mode is only Max or Min'
            t += 1
        self.weights = w
        self.time_steps = np.ones(len(self.arms)) 
        self.t = t
        
    def bandit_iter(self):
        weights = self.weights
        time_steps = self.time_steps
        t = self.t
        if self.mode == 'Max':
            u_ucb = weights - np.sqrt(1.5 * np.log(t)/time_steps)
        elif self.mode == 'Min':
            u_ucb = weights - np.sqrt(1.5 * np.log(t)/time_steps)
        else:
            raise 'Mode is only Max or Min'
        At = self.solver.solve(self.problem, lambda x, y: u_ucb[int(self.problem.arms2idx[(x, y)])])
        wAt = self.oracle.observe(self.problem, At)
        
        for idx, e in enumerate(At):
            arm = int(self.problem.arms2idx[e])
            weights[arm] = (time_steps[arm] * weights[arm] + wAt[idx]) / (time_steps[arm] + 1)
            time_steps[arm] += 1
        t += 1
        self.time_steps = time_steps
        self.weights = weights
        self.t = t        
        
    def solve(self, problem):
        weights = self.weights
        time_steps = self.time_steps
        t = self.t
        u_ucb = weights
        At = self.solver.solve(self.problem, lambda x, y: u_ucb[int(self.problem.arms2idx[(x, y)])])
        return At

In [34]:
sps = ShortestPathSolver()
spo = ShortestPathOracle()
At = sps.solve(shortest_path_problem, shortest_path_problem.get_weight)
np.sum(spo.observe(shortest_path_problem, At))

1.5690964126189044

In [35]:
b = CombUcb1(problem=shortest_path_problem, solver=sps, oracle=spo, mode='Min')

for i in range(20):
    b.bandit_iter()
    At = b.solve(shortest_path_problem)
    if i % 5 == 0:
        print np.sum(spo.observe(shortest_path_problem, At))
    

2.367257722050806
2.000071870781739
1.5690964126189044
1.5690964126189044


/home/halshater/anaconda/envs/gc_vision/lib/python2.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in log
/home/halshater/anaconda/envs/gc_vision/lib/python2.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in sqrt


In [36]:
#####################

In [37]:
import numpy as np
from pyomo import environ as pe


In [38]:
def solve_stylest_matching(stylist_workloads, clients, happiness_probabilities):
    model = pe.ConcreteModel()
    model.stylists = pe.Set(initialize=stylist_workloads.keys())
    model.clients = pe.Set(initialize=clients)
    model.happiness_probabilities = pe.Param(
        # On pyomo Set objects, the '*' operator returns the cartesian product
        model.stylists * model.clients,
        # The dictionary mapping (stylist, client) pairs to chances of a happy outcome
        initialize=happiness_probabilities,
        # Happiness probabilities are real numbers between 0 and 1
        within=pe.UnitInterval)

    model.stylist_workloads = pe.Param(
        model.stylists,
        initialize=stylist_workloads,
        within=pe.NonNegativeIntegers)
    model.assignments = pe.Var(
        # Defined over the client-stylist matrix
        model.stylists * model.clients,
        # Possible values are 0 and 1
        domain=pe.Binary)
    model.objective = pe.Objective(
        expr=pe.summation(model.happiness_probabilities, model.assignments),
        sense=pe.maximize)
    def respect_workload(model, stylist):
        # Count up all the clients assigned to the stylist
        n_clients_assigned_to_stylist = sum(
            model.assignments[stylist, client]
            for client in model.clients)
        # What's the max number of clients this stylist can work with?
        max_clients = model.stylist_workloads[stylist]
        # Make sure that sum is no more than the stylist's workload
        return n_clients_assigned_to_stylist <= max_clients

    model.respect_workload = pe.Constraint(
        # For each stylist in the set of all stylists...
        model.stylists,
        # Ensure that total assigned clients at most equal workload!
        rule=respect_workload)

    def one_stylist_per_client(model, client):
        # Count up all the stylists assigned to the client
        n_stylists_assigned_to_client = sum(
            model.assignments[stylist, client]
            for stylist in model.stylists)
        # Make sure that sum is equal to one
        return n_stylists_assigned_to_client == 1

    model.one_stylist_per_client = pe.Constraint(
        # For each client in the set of all clients...
        model.clients,
        # Ensure that exactly one stylist is assigned!
        rule=one_stylist_per_client)

    # Swap out "glpk" for "cbc" or "gurobi" if using another solver
    solver = pe.SolverFactory("glpk")
    # Add the keyword arg tee=True for a detailed trace of the solver's work.
    solution = solver.solve(model)
    return [i for i, j in model.assignments.get_values().items() if j == 1]

In [39]:
stylist_workloads = {
    "Alex": 1, "Jennifer": 2, "Andrew": 2,
    "DeAnna": 2, "Jesse": 3}

clients = (
    "Trista", "Meredith", "Aaron", "Bob", "Jillian",
    "Ali", "Ashley", "Emily", "Desiree", "Byron")

happiness_probabilities = dict(
    ((stylist, client), np.random.rand())
    for stylist in stylist_workloads
    for client in clients)


In [47]:
#solve_stylest_matching(stylist_workloads, clients, happiness_probabilities)

In [43]:
class StylesMatchingProblem:
    def __init__(self, stylist_workloads, clients, happiness_probabilities):
        self.stylist_workloads = stylist_workloads
        self.clients = clients
        self.happiness_probabilities = happiness_probabilities
        self.idx2arms = {str(i): j for i, j in enumerate(happiness_probabilities.keys())}
        self.arms2idx = {v: k for k, v in self.idx2arms.items()}
    def get_weight(x, y): 
        return self.happiness_probabilities[(x, y)]

In [45]:
class StylestMathcingSolver(Solver):
    def solve(self, problem, weights):
        return solve_stylest_matching(problem.stylist_workloads, problem.clients, problem.happiness_probabilities)
    
class StylestMathcingOracle(Oracle):
    def observe(self, problem, solution):
        return [problem.get_weight(x, y) for (x, y) in solution]


In [46]:
sp = StylesMatchingProblem()
sms = StylestMathcingSolver()
smo = StylestMathcingOracle()

At = sps.solve(shortest_path_problem, shortest_path_problem.get_weight)
np.sum(spo.observe(stylist_workloads, At))

1.5690964126189044

In [ ]:
b = CombUcb1(problem=shortest_path_problem, solver=sps, oracle=spo, mode='Min')

for i in range(20):
    b.bandit_iter()
    At = b.solve(shortest_path_problem)
    if i % 5 == 0:
        print np.sum(spo.observe(shortest_path_problem, At))
    

In [14]:
#!pip install git+https://github.com/Pyomo/pyomo

In [31]:
# assignments = model.assignments.get_values().items()
# for (stylist, client), assigned in sorted(assignments):
#     if assigned == 1:
#         print "{} will be styled by {}".format(client.rjust(8), stylist)
